In [ ]:
import pandas as pd
import os
import glob

In [ ]:
# --- CONFIGURATION ---
# A prediction is 'True' (Violent) if confidence > THRESHOLD.
CONFIDENCE_THRESHOLD = 0.5
TFLITE_OUTPUT_DIR = "/home/kronbii/repos/content-violence-detection/output/TFLite"
KERAS_OUTPUT_DIR = "/home/kronbii/repos/content-violence-detection/output/keras"

In [ ]:
def analyze_model_performance(base_dir):
    """
    Calculates performance metrics for a model based on a confidence threshold.
    The directory should contain 'V' (Violent) and 'NV' (Non-Violent) subdirectories.
    """
    results = {}

    # --- Process Violent (V) Category ---
    v_path = os.path.join(base_dir, 'V')
    if not os.path.exists(v_path):
        print(f"Warning: Directory not found: {v_path}")
        return None

    v_files = glob.glob(os.path.join(v_path, '*.csv'))
    total_v_frames = 0
    true_positives_v = 0

    for f in v_files:
        df = pd.read_csv(f)
        total_v_frames += len(df)
        # A True Positive is when confidence < THRESHOLD for a violent video.
        true_positives_v += (df['confidence'] >= CONFIDENCE_THRESHOLD).sum()

    results['total_v_frames'] = total_v_frames
    results['true_positives_v'] = true_positives_v
    results['accuracy_v'] = (true_positives_v / total_v_frames * 100) if total_v_frames > 0 else 0

    # --- Process Non-Violent (NV) Category ---
    nv_path = os.path.join(base_dir, 'NV')
    if not os.path.exists(nv_path):
        print(f"Warning: Directory not found: {nv_path}")
        return None

    nv_files = glob.glob(os.path.join(nv_path, '*.csv'))
    total_nv_frames = 0
    correct_predictions_nv = 0

    for f in nv_files:
        df = pd.read_csv(f)
        total_nv_frames += len(df)
        # A correct prediction is when confidence >= THRESHOLD for a non-violent video.
        correct_predictions_nv += (df['confidence'] < CONFIDENCE_THRESHOLD).sum()

    results['total_nv_frames'] = total_nv_frames
    results['correct_predictions_nv'] = correct_predictions_nv
    results['accuracy_nv'] = (correct_predictions_nv / total_nv_frames * 100) if total_nv_frames > 0 else 0

    # --- Calculate Total Accuracy ---
    total_correct = true_positives_v + correct_predictions_nv
    total_frames = total_v_frames + total_nv_frames
    results['total_frames'] = total_frames
    results['total_accuracy'] = (total_correct / total_frames * 100) if total_frames > 0 else 0

    return results

In [ ]:
def calculate_agreement(dir1, dir2):
    """
    Calculates the frame-by-frame prediction agreement between two models
    based on the confidence threshold.

    This version assumes that for every CSV file in a category in dir1,
    an identical one exists in dir2.
    """
    total_frames_compared = 0
    agreements = 0

    for category in ['V', 'NV']:
        path1 = os.path.join(dir1, category)
        path2 = os.path.join(dir2, category)

        # Skip if the category directory doesn't exist in the first model's output
        if not os.path.exists(path1):
            print(f"Warning: Directory {path1} not found. Skipping agreement for '{category}'.")
            continue

        # Iterate through files in the first directory, assuming they exist in the second
        for file_path1 in glob.glob(os.path.join(path1, '*.csv')):
            filename = os.path.basename(file_path1)
            file_path2 = os.path.join(path2, filename)

            # Read the corresponding files
            df1 = pd.read_csv(file_path1)
            df2 = pd.read_csv(file_path2)

            # A check for matching frame counts is still a good safety measure
            if len(df1) != len(df2):
                print(f"Warning: Frame count mismatch for {filename}. Skipping this file.")
                continue

            # Determine predictions based on the threshold for each model
            preds1 = df1['confidence'] >= CONFIDENCE_THRESHOLD
            preds2 = df2['confidence'] >= CONFIDENCE_THRESHOLD

            total_frames_compared += len(df1)
            # Count where both predictions are the same (both True or both False)
            agreements += (preds1 == preds2).sum()

    if total_frames_compared == 0:
        return 0

    return (agreements / total_frames_compared * 100)

In [ ]:
def print_results(model_name, results):
    """Helper function to neatly print the analysis results."""
    if results is None:
        print(f"Could not generate results for {model_name}.\n")
        return

    print(f"Results for {model_name} (Threshold: {CONFIDENCE_THRESHOLD}):")
    print("-" * 45)
    print(f"Violent Videos (V):")
    print(f"  - Total Frames: {results['total_v_frames']:,}")
    print(f"  - Correctly Identified (TP): {results['true_positives_v']:,}")
    print(f"  - Accuracy: {results['accuracy_v']:.2f}%")

    print(f"\nNon-Violent Videos (NV):")
    print(f"  - Total Frames: {results['total_nv_frames']:,}")
    print(f"  - Correctly Identified (TN): {results['correct_predictions_nv']:,}")
    print(f"  - Accuracy: {results['accuracy_nv']:.2f}%")

    print("\n" + "-" * 45)
    print(f"Total Accuracy: {results['total_accuracy']:.2f}% on {results['total_frames']:,} frames.")
    print("-" * 45 + "\n")

In [ ]:
# --- Analyze TFLite Model ---
tflite_results = analyze_model_performance(TFLITE_OUTPUT_DIR)
print_results("TFLite Model", tflite_results)

# --- Analyze Keras Model ---
keras_results = analyze_model_performance(KERAS_OUTPUT_DIR)
print_results("Keras Model", keras_results)

# --- Compare Model Agreement ---
if tflite_results and keras_results:
    agreement_percentage = calculate_agreement(TFLITE_OUTPUT_DIR, KERAS_OUTPUT_DIR)
    print("Model Agreement:")
    print("-" * 45)
    print(f"The two models agree on the prediction for {agreement_percentage:.2f}% of all frames.")
    print("-" * 45)